# Make dhlab db

Make a fulltext table dhlab style

In [2]:
import dhlab as dh
from pymongo import MongoClient
import pandas as pd
from dataclasses import dataclass
import sqlite3
import matplotlib.pyplot as plt

db = MongoClient().norn # Connect to database
tok = dh.nbtokenizer.tokenize # Tokenizer
collection = db['poems']

In [ ]:
# Add new tokenized field to database
for doc in collection.find():
    tokens = tok(doc['text'])
    collection.update_one({'_id': doc['_id']}, {'$set': {'tokens': tokens}})

In [59]:
# Parse mongodb to dataclasses

@dataclass
class Metadata:
    _id : str
    urn: str
    title: str

@dataclass
class Poem:
    _id: str
    tokens: list
    
    @property
    def count(self):
        return list(range(1, len(self.tokens) + 1))
    
metadata_list = []
poem_list = []

for doc in db.poems.find():
    doc['tokens'] = tok(doc['text'])
    metadata_list.append(Metadata(str(doc['_id']), doc['urn'], doc['title']))
    poem_list.append(Poem(str(doc['_id']), doc['tokens']))

In [60]:
# Create token database
poem_list_res = []
for poem in poem_list:
    for token, count in zip(poem.tokens, poem.count):
        poem_list_res.append((poem._id, token, count))



with sqlite3.connect("data.db") as con:
    con.execute("DROP TABLE IF EXISTS metadata")
    con.execute("DROP TABLE IF EXISTS poems")
    
    con.execute("CREATE TABLE metadata (_id, urn, title, PRIMARY KEY(_id))")
    con.execute("CREATE TABLE poems (_id, tokens, count, FOREIGN KEY(_id) REFERENCES metadata(_id))")
    
    con.executemany("INSERT INTO metadata VALUES (?, ?, ?)", [(x._id, x.urn, x.title) for x in metadata_list])
    # con.executemany("INSERT INTO poems VALUES (?, ?, ?)", [(x._id, x.tokens, x.count) for x in poem_list])
    con.executemany("INSERT INTO poems VALUES (?, ?, ?)", poem_list_res)
    

In [77]:
# All texts
query = """SELECT p._id, title, tokens, count(*) FROM poems p JOIN metadata m on m._id=p._id GROUP BY p._id, tokens"""

with sqlite3.connect("data.db") as con:
    res = con.execute(query).fetchall()

In [102]:
# Single token
query = """SELECT p._id, title, tokens, count(*) FROM poems p JOIN metadata m on m._id=p._id  WHERE tokens='.' GROUP BY p._id, tokens"""

with sqlite3.connect("data.db") as con:
    res = con.execute(query).fetchall()



In [99]:
# Single texts
query = """SELECT p._id, title, tokens, count(*) FROM poems p JOIN metadata m on m._id=p._id WHERE title='Over vore to smaa døde' GROUP BY p._id, tokens"""

with sqlite3.connect("data.db") as con:
    res = con.execute(query).fetchall()

In [103]:
df = pd.DataFrame(res, columns = ['_id', 'title', 'tokens', 'count'])

df.sort_values(by = 'count', ascending = False).head(20)

,_id,title,tokens,count
2283,65a550fb4a06fb61333ed599,Sæterliv,.,246
653,65a550fa4a06fb61333ecf2d,Ved Aarsskiftet,.,198
265,65a550fa4a06fb61333ecda3,Slottet,.,185
562,65a550fa4a06fb61333eced2,Sirius som Séer,.,166
553,65a550fa4a06fb61333ecec9,Sidste nat,.,149
1411,65a550fa4a06fb61333ed226,Sangerstevnet,.,145
2241,65a550fb4a06fb61333ed56f,Fossefaldet,.,139
2388,65a550fb4a06fb61333ed603,Dommedag,.,137
2231,65a550fb4a06fb61333ed565,Paa Hospitalet,.,134
560,65a550fa4a06fb61333eced0,Tredie nat,.,125
